In [1]:
import pandas as pd
from langdetect import detect
from googletrans import Translator

In [2]:
import nltk
#nltk.download('vader_lexicon')

In [3]:
!where python  # On Windows

C:\Users\zhiyu\AppData\Local\Programs\Python\Python312\python.exe
C:\Users\zhiyu\AppData\Local\Continuum\anaconda3\python.exe
C:\Users\zhiyu\AppData\Local\Microsoft\WindowsApps\python.exe


INFO: Could not find "#".
INFO: Could not find "On".
INFO: Could not find "Windows".


In [4]:
df = pd.read_csv('dbt_sqlite_practice/data/listings.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dbt_sqlite_practice/data/listings.csv'

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df[df['host_id'] == 6774][['id','last_scraped','host_id','host_name', 'host_since', 'host_location', 'host_about','host_response_time', 'host_response_rate','host_listings_count','availability_30', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value',]]

In [ ]:
df['neighbourhood_cleansed'].unique()

In [5]:
df_reviews = pd.read_csv('dbt_sqlite_practice/seeds-data/reviews.csv')

In [6]:
len(df_reviews)

1887519

In [7]:
df_reviews.head()

,listing_id,date
0,13913,2010-08-18
1,13913,2011-07-11
2,13913,2011-09-13
3,13913,2011-10-03
4,13913,2011-10-09


In [8]:
df_reviews_detailed = pd.read_csv('dbt_sqlite_practice/seeds-data/reviews_detailed.csv')

In [9]:
df_reviews_detailed.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [10]:
df_reviews_detailed['date'].unique()

array(['2010-08-18', '2011-07-11', '2011-09-13', ..., '2024-09-09',
       '2020-06-18', '2024-09-10'], dtype=object)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
%%time 
for i in df_reviews_detailed.head():
    print(i)
    #sia.polarity_scores(i['comments'])

In [ ]:
def analyze_sentiment(comment):
    local_sia = SentimentIntensityAnalyzer()
    local_translator = Translator()
    if isinstance(comment,str):
        try:
            language_detect = detect(comment)
            if language_detect != 'en':
                comment = local_translator.translate(comment, src=language_detect, dest='en').text
                sentiment_language['translated_comment'] = comment
            sentiment_language = local_sia.polarity_scores(comment)
            sentiment_language['language'] = language_detect
            return sentiment_language
        except Exception as e:
            print(f'Translation failed: {e}')
            pass
    else:
        pass

In [ ]:
tqdm.pandas()

In [ ]:
sentiment_scores_test = df_reviews_detailed.head(100)['comments'].progress_apply(analyze_sentiment).apply(pd.Series)

In [ ]:
sentiment_scores_test

In [ ]:
df_reviews_detailed_test = pd.concat([df_reviews_detailed.head(100), sentiment_scores_test.rename(columns={'pos': 'positive', 'neu': 'neutral', 'neg': 'negative'})], axis=1)

In [ ]:
df_reviews_detailed_test[df_reviews_detailed_test['language']!='en']

In [ ]:
sentiment_scores = df_reviews_detailed['comments'].progress_apply(analyze_sentiment).apply(pd.Series)

In [ ]:
df_reviews_detailed = pd.concat([df_reviews_detailed, sentiment_scores.rename(columns={'pos': 'positive', 'neu': 'neutral', 'neg': 'negative'})], axis=1)

In [ ]:
df_reviews_detailed[df_reviews_detailed['negative']>0.1].head()

In [ ]:
df_reviews_detailed[df_reviews_detailed['id']==1314781]['comments']

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_reviews_detailed[df_reviews_detailed['id']==1314781]['comments'].values[0]

In [ ]:
str(df_reviews_detailed[df_reviews_detailed['id']==1314781]['comments'].values)

In [ ]:
detect(df_reviews_detailed[df_reviews_detailed['id']==1314781]['comments'].values[0])

In [ ]:
google_translator = Translator()

In [ ]:
print(google_translator.translate(df_reviews_detailed[df_reviews_detailed['id']==1314781]['comments'].values[0]).text)

In [ ]:
google_translator.translate('El departamento esta muy bien ubicado y equipado.\r<br/>La persona que nos recibio, fue muy amable en indicarnos donde ir a comprar comida en el horario que llegamos. Y Philipa nos permitio dejar nuestras valijas mas tarde del horario de salida.\r<br/>Tiene un gran defecto que es el ruido de camiones en la semana, muy temprano por la mañana, ya que hay un negocio de muebles a la vuelta y los camiones estacionan justo en la calle a la que da la habitacion y hacen mucho ruido.\r<br/>El sillon cama no es muy comodo para descansar.\r<br/>Me resulto algo incomodo que Philippa llegue al departamento sin aviso previo, y que revise todo, nunca me paso algo asi, no se si es algo comun en Londres, pero he alquilado mas departamentos en otras ciudades, y eso me resulto algo molesto y extraño').text

In [ ]:
detect('El departamento esta muy bien ubicado y equipado.\r<br/>La persona que nos recibio, fue muy amable en indicarnos donde ir a comprar comida en el horario que llegamos. Y Philipa nos permitio dejar nuestras valijas mas tarde del horario de salida.\r<br/>Tiene un gran defecto que es el ruido de camiones en la semana, muy temprano por la mañana, ya que hay un negocio de muebles a la vuelta y los camiones estacionan justo en la calle a la que da la habitacion y hacen mucho ruido.\r<br/>El sillon cama no es muy comodo para descansar.\r<br/>Me resulto algo incomodo que Philippa llegue al departamento sin aviso previo, y que revise todo, nunca me paso algo asi, no se si es algo comun en Londres, pero he alquilado mas departamentos en otras ciudades, y eso me resulto algo molesto y extraño')

## Pyspark implementation

In [11]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\zhiyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\pyspark'

In [12]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, FloatType, StructType, StructField

In [13]:
from pyspark.sql import SparkSession

In [14]:
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_202"  # Replace with your Java 8 path
os.environ["PATH"] = os.environ["JAVA_HOME"] + "\\bin;" + os.environ["PATH"]

In [15]:
sparks = SparkSession.builder.appName('TestSparkSess').getOrCreate()

Exception: Java gateway process exited before sending its port number

In [ ]:
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

In [ ]:
spark_df = sparks.createDataFrame(data)

In [ ]:
spark_df.show()

In [ ]:
# Pandas to Spark
df_reviews_detailed_sp = sparks.createDataFrame(df_reviews_detailed)

In [ ]:
df_reviews_detailed_sp = sparks.read.format("csv").options(header="true").load("dbt_sqlite_practice/seeds-data/reviews_detailed.csv")

In [ ]:
df_reviews_detailed_sp.show(5)

In [ ]:
# define schema
schema = StructType([
    StructField("language", StringType(), True),
    StructField("translated_comment", StringType(), True),
    StructField("pos", FloatType(), True),
    StructField("neu", FloatType(), True),
    StructField("neg", FloatType(), True),
    StructField("compound", FloatType(), True),
    
])

In [ ]:
# Register UDF

detect_sentiment_udf = udf(analyze_sentiment, schema)

In [ ]:
df_reviews_detailed_sp = df_reviews_detailed_sp.withColumn("sentiment", detect_sentiment_udf(col("comments")))

In [ ]:
# Split the `sentiment` struct column into separate columns
df_reviews_detailed_sp = df_reviews_detailed_sp.select(
    col("listing_id"),
    col("id"),
    col("date"),
    col("reviewer_id"),
    col("reviewer_name"),
    col("comments"),
    col("sentiment.language").alias("language"),
    col("sentiment.translated_comment").alias("translated_comment"),
    col("sentiment.pos").alias("pos"),
    col("sentiment.neu").alias("neu"),
    col("sentiment.neg").alias("neg"),
    col("sentiment.compound").alias("compound")
)


In [ ]:
df_reviews_detailed_sp.show(5)

In [ ]:
non_eng_reviews = df_reviews_detailed_sp.filter(col("language") != "en")

In [ ]:
print((non_eng_reviews.count(), len(non_eng_reviews.columns)))

In [ ]:
df_reviews_detailed_sp.explain(True)

In [ ]:
import os
print(os.system("java -version"))

In [ ]:
print(os.system("java -version"))

In [ ]:
print(os.environ["JAVA_HOME"])

In [ ]:
print(os.environ["PATH"])

In [ ]:
# Write a script that will partition into a single day, per file and write into each year-month folder and place in a raw_daata folder
# Write py script that does the analysis and outputs the data into a landing area for dbt
# Udate the dbt scripts so that they ingest incrementally
# Create a airflow orchestrator that can orchestrate the above script and another one to execute dbt run to refresh data
# dbt ingest the